# 共通処理

In [ ]:
import os
import sys

import h5py

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:
import json

try:
    with open('setting.json', 'r') as file:
        setting = json.load(file)
except FileNotFoundError:
    print("先にset_run.ipynbで解析runを設定してください")

hdf5_path = setting['hdf5_path']
hdf5_path


In [ ]:
from model.File.HDF5 import HDF5

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# データを取り出す準備

In [ ]:
# hdf5内のlayer pathを取得する
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)
    print("")
    to_spe = hdf.search_data_path(query='spe_path')
    to_calib = hdf.search_data_path(query='calib_path')
    to_dist = hdf.search_data_path(query='dist_path')

In [ ]:
# それぞれのファイルのpathを取得する
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)
    print("")
    spe_path = hdf.return_data(data_path=to_spe)
    calib_path = hdf.return_data(data_path=to_calib)
    dist_path = hdf.return_data(data_path=to_dist)

# データを取り出して格納する

## Laser

In [ ]:
# クラスのimportとインスタンスの作成
from model.Temperature.Laser import LaserProfile
laser = LaserProfile()

In [ ]:
# 設定したLaserパラメータを取得して確認
with open('setting.json', 'r') as file:
    setting = json.load(file)

laser_params = setting['laser_params']
laser_params

In [ ]:
# 設定して配列を作成する
params_list = list(laser_params.keys())

for param in params_list:
    exec(f"laser.{param} = {laser_params[param]}")
    exec(f"print(laser.{param})")
laser.get_laser_profile_arr()

# plotして確認しておく
plt.plot(laser.time_arr, laser.power_arr)
plt.show()
plt.close()

In [ ]:
# hdf5に書き込む
laser_layer = 'entry/laser/'

with h5py.File(hdf5_path, 'a') as f_append:
    f_append.create_dataset(name=laser_layer + 'time_arr', data=laser.time_arr)
    f_append.create_dataset(name=laser_layer + 'power_arr', data=laser.power_arr)

In [ ]:
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

## dist

In [ ]:
# クラスのimportとインスタンスの作成
from model.Temperature.T_dist import T_dist
dist = T_dist(filepath=dist_path)

In [ ]:
# 温度データを整形する
dist.set_T_data()

In [ ]:
# plotして確認しておく
plt.imshow(dist.temp_df, aspect='auto')
plt.show()
plt.close()

In [ ]:
# hdf5に書き込む
dist_layer = 'entry/dist/'

with h5py.File(hdf5_path, 'a') as f_append:
    f_append.create_dataset(name=dist_layer + 'T_arr', data=dist.temp_df.values)

In [ ]:
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

## spe

In [ ]:
# クラスのimportとインスタンスの作成
from model.Temperature.Spe import SpeFile
spe = SpeFile(filepath=spe_path)

In [ ]:
# あると便利そうな配列を作る
spe.time_arr = np.arange(spe.num_frames) / spe.framerate
spe.pixel_arr = np.arange(spe.roi_height)

In [ ]:
# hdf5に書き込む
spe_layer = 'entry/spe/'

with h5py.File(hdf5_path, 'a') as f_append:
    f_append.create_dataset(name=spe_layer + 'filename',         data=spe.filename)
    f_append.create_dataset(name=spe_layer + 'frame_num',        data=spe.num_frames)
    f_append.create_dataset(name=spe_layer + 'exposure_ms',      data=spe.exposure_time)
    f_append.create_dataset(name=spe_layer + 'fps',              data=spe.framerate)
    f_append.create_dataset(name=spe_layer + 'OD',               data=spe.OD)
    f_append.create_dataset(name=spe_layer + 'date',             data=spe.date)
    f_append.create_dataset(name=spe_layer + 'calibration_date', data=spe.calibration_date)
    f_append.create_dataset(name=spe_layer + 'img',              data=spe.img)
    f_append.create_dataset(name=spe_layer + 'wl_arr',           data=spe.x_calibration)
    f_append.create_dataset(name=spe_layer + 'time_arr',         data=spe.time_arr)
    f_append.create_dataset(name=spe_layer + 'pixel_arr',        data=spe.time_arr)

In [ ]:
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

## calib

In [ ]:
# クラスのimportとインスタンスの作成
from model.Temperature.T_calib import T_calib
calib = T_calib(filepath=calib_path)

In [ ]:
# スペクトルデータを取得する
calib_img = calib.return_calib_img(frame_num=spe.num_frames)

In [ ]:
# hdf5に書き込む
calib_layer = 'entry/calib/'

with h5py.File(hdf5_path, 'a') as f_append:
    f_append.create_dataset(name=calib_layer + 'spectra', data=calib_img)

In [ ]:
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    # HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)
    hdf = HDF5(f=f_read)
    img = hdf.return_data(data_path='entry/calib/spectra')
    img.shape